# Анализ базы данных крупного сервиса для чтения книг по подписке

## Описание проекта

Наша компания решила быть на волне и купила крупный сервис для чтения книг по подписке. Наша первая задача как аналитика — проанализировать базу данных. В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

## Цель исследования

Изучить данные о книгах, авторах, издательствах, рейтингах и обзорах и ответить на поставленные вопросы

## Исследование таблиц — вывод первых строк

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import text, create_engine

#Чтобы при выводе таблицы отображался весь столбец с описанием
pd.set_option('display.max_colwidth', 0)

In [2]:
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных

connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

In [3]:
# чтобы выполнить SQL-запрос, используем Pandas
query = '''SELECT * FROM books LIMIT 5'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope #1),322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before Columbus,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [4]:
# Таблица authors
query = '''SELECT * FROM authors LIMIT 5'''
pd.io.sql.read_sql(sql=text(query), con = con)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [5]:
# Таблица publishers 
query = '''SELECT * FROM  publishers  LIMIT 5'''
pd.io.sql.read_sql(sql=text(query), con = con)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [6]:
# Таблица ratings 
query = '''SELECT * FROM   ratings  LIMIT 5'''
pd.io.sql.read_sql(sql=text(query), con = con)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [7]:
# Таблица reviews 
query = '''SELECT * FROM  reviews  LIMIT 5'''
pd.io.sql.read_sql(sql=text(query), con = con)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. Over provide race technology continue these.
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Among admit investment argue security.
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but person sport treatment industry. Kitchen decision deep the. Social party body the.
3,4,3,johnsonamanda,Finally month interesting blue could nature cultural bit. Prepare beat finish grow that smile teach. Dream me play near.
4,5,3,scotttamara,Nation purpose heavy give wait song will. List dinner another whole positive radio fast. Music staff many green.


Мы познакомились с таблицами, убедились, что все столбцы на месте.

## Задания

### Посчитаем, сколько книг вышло после 1 января 2000 года;

In [8]:
query = '''
SELECT COUNT(book_id)
FROM books  
WHERE publication_date > '2000-01-01'
'''

pd.io.sql.read_sql(sql=text(query), con = con)

,count
0,819


С 1 января 2000 года вышло 819 книг.

### Для каждой книги посчитаем количество обзоров и среднюю оценку;

In [9]:
query = '''
SELECT b.book_id, 
       b.title, 
       COUNT(DISTINCT rev.review_id) AS count_review, 
       ROUND(AVG(r.rating),3) AS avg_rating
FROM books AS b 
LEFT JOIN reviews AS rev ON b.book_id = rev.book_id
LEFT JOIN ratings AS r ON b.book_id = r.book_id
GROUP BY b.book_id, 
         b.title
ORDER BY count_review DESC,
         avg_rating DESC   

'''

pd.io.sql.read_sql(sql=text(query), con = con)

,book_id,title,count_review,avg_rating
0,948,Twilight (Twilight #1),7,3.663
1,302,Harry Potter and the Prisoner of Azkaban (Harry Potter #3),6,4.415
2,299,Harry Potter and the Chamber of Secrets (Harry Potter #2),6,4.288
3,656,The Book Thief,6,4.264
4,734,The Glass Castle,6,4.207
...,...,...,...,...
995,191,Disney's Beauty and the Beast (A Little Golden Book),0,4.000
996,387,Leonardo's Notebooks,0,4.000
997,221,Essential Tales and Poems,0,4.000
998,83,Anne Rice's The Vampire Lestat: A Graphic Novel,0,3.667


Всего у нас 1000 книг, самое большое кол-во обзоров на книгу 7 (Twilight (Twilight #1)), самое маленькое 0 (у многих книг). По таблице можем увидеть, что средняя оценка не зависит от количества обзоров на книгу.

### Определим издательство, которое выпустило наибольшее число книг толще 50 страниц — так мы исключим из анализа брошюры;

In [10]:
query = '''
SELECT b.publisher_id, 
       p.publisher, 
       COUNT(*) AS count_books
FROM books AS b
LEFT JOIN publishers AS p ON b.publisher_id = p.publisher_id
WHERE num_pages > 50
GROUP BY b.publisher_id,
         p.publisher
ORDER BY COUNT(*) DESC
LIMIT 1
        '''
pd.io.sql.read_sql(sql=text(query), con = con)

,publisher_id,publisher,count_books
0,212,Penguin Books,42


Penguin Books - издательство, которое выпустило наибольшее число книг толще 50 страниц, таких книг у этого издательства 42.

### Определим автора с самой высокой средней оценкой книг — будем учитывайть только книги с 50 и более оценками;

In [11]:
query = '''
SELECT a.author ,
       AVG(r.rating) AS avg_rating
FROM ratings AS r
INNER JOIN books AS b ON b.book_id = r.book_id
INNER JOIN authors AS a ON b.author_id = a.author_id
WHERE r.book_id IN 
(
       SELECT book_id
       FROM ratings
       GROUP BY book_id
       HAVING COUNT(book_id)>=50
       )
GROUP BY a.author_id
ORDER BY avg_rating DESC
LIMIT 1
        '''
pd.io.sql.read_sql(sql=text(query), con = con)

,author,avg_rating
0,J.K. Rowling/Mary GrandPré,4.287097


Автор серии романов о Гарри Поттере - Дж. К. Роулинг - автор с самой высокой средней оценкой книг (средняя оценка 4,3). Мэри ГрандПре — американский иллюстратор, наиболее известная своими иллюстрациями обложек и глав книг о Гарри Поттере в их американских изданиях, опубликованных Scholastic. В целом, нечему удивляться, книги о Гарри Поттере, наверное, одни из самых популярных книг в мире, их интересно читать и взрослым и детям.

### Посчитаем среднее количество обзоров от пользователей, которые поставили больше 48 оценок.

In [12]:
query = '''
SELECT COUNT(DISTINCT rev.review_id)/ COUNT(DISTINCT rev.username) AS avg_review
FROM reviews AS rev
INNER JOIN ratings AS r ON rev.username = r.username
WHERE rev.username IN 
(
       SELECT username
       FROM ratings
       GROUP BY username
       HAVING COUNT(rating_id)>50
       )

        '''
pd.io.sql.read_sql(sql=text(query), con = con)

,avg_review
0,24


Пользователи, которые поставили больше 48 оценок, в среднем оставили 24 обзора на книги.

## Вывод

Мы проанализировали базу данных и узнали, что:

* с 1 января 2000 года вышло 819 книг
* кол-во обзоров на книгу находится в диапазоне от 0 до 7, всего 1000 книг, при этом, средняя оценка не зависит от количества обзоров на книгу
* Penguin Booksи - здательство, которое выпустило 42 книги толще 50 страниц
* автор с самой высокой средней оценкой книг - Дж. К. Роулинг (в соавторстве с иллюстратором Мэри ГрандПре)
* активные пользователи (пользователи, которые поставили больше 48 оценок) в среднем написали 24 обзора на книги.